In [1]:
import pandas as pd
import numpy as np
import pymorphy2

In [2]:
df = pd.read_excel('augmented_sample.xlsx')

In [3]:
morph = pymorphy2.MorphAnalyzer()
morph.parse('чем')[0].tag.POS 

'CONJ'

In [4]:
add_dict = {'verb':['как ', 'хочу ', 'что нужно для того, чтобы '], 
 'noun':['как получить ', 'оформить ', 'сделать ', 'заказать '],
'заявление':['как подать ', 'бланк ']}

In [5]:
stop_words = ['в', 'и', 'на', 'что', 'для', 'об', 'чтобы', 'и/или']

In [6]:
target_copy = []
anchors = []
zeros = []
for row in df.iterrows():
    zero_anchs = list(df[df['категория'] != row[1][1]].sample(frac=1)['вопрос'])
    anchors.extend(zero_anchs)
    target_copy.extend([row[1][0]]*len(zero_anchs))
    zeros.extend([0]*len(zero_anchs))

In [7]:
anchs = []
targs = []
ones = []
for row in df.iterrows():
    req = [term for term in row[1][0].rstrip('?').split() if term not in stop_words]
    if req[0] in ('Готовность', 'статус'):
        anch = 'как узнать ' + row[1][0]
        anchs.extend([anch, row[1][0]])
        targs.extend([row[1][0]]*2)
        ones.extend([1]*2)
    elif row[1][1] in ('информация УО', 'отсрочка от армии'):
        anchs.append(row[1][0])
        targs.append(row[1][0])
        ones.append(1)
    else:
        if row[1][1] in ['мои документы', 'студенты', 'коммерция', 'бывшие студенты', 'сотрудники', 'Пропускная система']:
            parts_of_speech = list(map(lambda x: morph.parse(x)[0].tag.POS, req))
            if ('VERB' in parts_of_speech or 'INFN' in parts_of_speech) and 'CONJ' in parts_of_speech or sum(list(map(lambda x: x in ('VERB', 'INFN'), parts_of_speech))) > 1:
#                 print(row[1][0], end='conj\n')
                if 'ADJF' in parts_of_speech:
                    idx = min(parts_of_speech.index('ADJF'), parts_of_speech.index('NOUN'))
                else:
                    idx = parts_of_speech.index('NOUN')
                anchor = ' '.join(row[1][0].split()[idx:])
                anchs.extend([anchor, row[1][0]])
                targs.extend([row[1][0]]*2)
                ones.extend([1]*2)
            elif ('VERB' in parts_of_speech or 'INFN' in parts_of_speech) and 'CONJ' not in parts_of_speech:
#                 print(row[1][0], end=' verb\n')
                anchor = [add + row[1][0] for add in add_dict['verb']]
                anchor.append(row[1][0])
                anchs.extend(anchor)
                targs.extend([row[1][0]]*len(anchor))
                ones.extend([1]*len(anchor))
            else:
#                 print(row[1][0])
                anchor = [add + row[1][0] for add in add_dict['noun']]
                anchor.append(row[1][0])
                anchs.extend(anchor)
                targs.extend([row[1][0]]*len(anchor))
                ones.extend([1]*len(anchor))
#                 print(*anchor, sep='\n')
        if row[1][1] == 'заявления':
            if req[0] in ['Заявление', 'Мотивационное', 'Объяснительная', 'Согласие', 'Внесение']:
    #             print(row[1][0])
                anchor = [add + row[1][0] for add in add_dict['заявление']]
                anchor.append(row[1][0])
                anchs.extend(anchor)
                targs.extend([row[1][0]]*len(anchor))
                ones.extend([1]*len(anchor))
    #             print(*anchor, sep='\n')
            elif req[0] == 'Характеристика':
                anchor = [add + row[1][0] for add in add_dict['noun']]
                anchor.append(row[1][0])
                anchs.extend(anchor)
                targs.extend([row[1][0]]*len(anchor))
                ones.extend([1]*len(anchor))
#                 print(*anchor, sep='\n')


In [8]:
pd.DataFrame({'target':targs+target_copy, 'anchor':anchs+anchors, 'score':ones+zeros}).sample(frac=1).to_excel('train_sample1.xlsx', header=True, index=False)